In [10]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Filepaths

In [2]:
rootpath = '/media/sf_VBox_Shared/London/raw/first5/'
annotations_path = rootpath + 'tud_first5.csv'
wearcodes_path = rootpath + 'wearcodes.csv'

In [59]:
filepath = rootpath + 'accelerometer_5second/'

## Functions

In [60]:
def loadData(name):
    result = {}
    for day in ['1', '2']:
        #try:
            filename = name + '_day' + day + '.csv'
            filename = filepath + filename
            data = pd.read_csv(filename, index_col='timestamp', parse_dates=True)
            result[(name, day)] = data
        #except:
        #    print 'File not found: ',name
        #    return None
    return result

In [ ]:
def parse_time(date_string):
    return datetime.datetime.strptime(s, '%Y-%m-%dT%H:%M:%S.%fz')


## Load and preprocess annotations

In [16]:
annotations = pd.read_csv(annotations_path)
print(annotations.shape)
annotations.head()

(1440, 5)


,serflag,start_time,end_time,act,act_label
0,52920071_100,2015-03-01T04:00:00.000Z,2015-03-01T04:10:00.000Z,1.0,Sleeping and resting (including sick in bed)
1,52920071_100,2015-03-01T04:10:00.000Z,2015-03-01T04:20:00.000Z,1.0,Sleeping and resting (including sick in bed)
2,52920071_100,2015-03-01T04:20:00.000Z,2015-03-01T04:30:00.000Z,1.0,Sleeping and resting (including sick in bed)
3,52920071_100,2015-03-01T04:30:00.000Z,2015-03-01T04:40:00.000Z,1.0,Sleeping and resting (including sick in bed)
4,52920071_100,2015-03-01T04:40:00.000Z,2015-03-01T04:50:00.000Z,1.0,Sleeping and resting (including sick in bed)


In [17]:
annotations['start_time'] = [parse_time(s) for s in annotations['start_time']]
annotations['end_time'] = [parse_time(s) for s in annotations['end_time']]
annotations.head()

,serflag,start_time,end_time,act,act_label
0,52920071_100,2015-03-01 04:00:00,2015-03-01 04:10:00,1.0,Sleeping and resting (including sick in bed)
1,52920071_100,2015-03-01 04:10:00,2015-03-01 04:20:00,1.0,Sleeping and resting (including sick in bed)
2,52920071_100,2015-03-01 04:20:00,2015-03-01 04:30:00,1.0,Sleeping and resting (including sick in bed)
3,52920071_100,2015-03-01 04:30:00,2015-03-01 04:40:00,1.0,Sleeping and resting (including sick in bed)
4,52920071_100,2015-03-01 04:40:00,2015-03-01 04:50:00,1.0,Sleeping and resting (including sick in bed)


In [24]:
# Check if end > start
differences = (annotations['end_time'] - annotations['start_time'] ) 
print('Number none: ', sum(differences.isnull()))

('Number none: ', 0)


In [37]:
# look at the ones that are not 10 minutes
annotations[differences != datetime.timedelta(minutes=10)]

,serflag,start_time,end_time,act,act_label
119,52920071_100,2015-03-01 23:50:00,2015-03-02,1.0,Sleeping and resting (including sick in bed)
263,52920071_100,2015-03-03 23:50:00,2015-03-04,1.0,Sleeping and resting (including sick in bed)
407,62980071_100,2015-03-15 23:50:00,2015-03-16,43.0,Reading (not for school)
551,62980071_100,2015-03-16 23:50:00,2015-03-17,2.0,"Personal care (including taking a shower/bath,..."
695,81510041_100,2015-03-11 23:50:00,2015-03-12,1.0,Sleeping and resting (including sick in bed)
839,81510041_100,2015-03-15 23:50:00,2015-03-16,1.0,Sleeping and resting (including sick in bed)
983,102650051_100,2015-03-05 23:50:00,2015-03-06,1.0,Sleeping and resting (including sick in bed)
1127,102650051_100,2015-03-08 23:50:00,2015-03-09,1.0,Sleeping and resting (including sick in bed)
1271,122650071_100,2015-03-06 23:50:00,2015-03-07,1.0,Sleeping and resting (including sick in bed)
1415,122650071_100,2015-03-07 23:50:00,2015-03-08,1.0,Sleeping and resting (including sick in bed)


In [38]:
# It seems that we need to adjust the end times for these with one day
annotations.loc[differences != datetime.timedelta(minutes=10), 'end_time'] +=  datetime.timedelta(days=1)
sum( (annotations['end_time'] - annotations['start_time'] ) != datetime.timedelta(minutes=10))

10

## Join with wearcodes

In [39]:
wearcodes = pd.read_csv(wearcodes_path)
print(wearcodes.shape)
wearcodes.head()

(5, 8)


,Monitor,Day1,Day2,binFile,serflag,fileSize,day1Ready,day2Ready
0,16566,11/03/2015,15/03/2015,__016566_2015-03-24 14-52-30.bin,81510041_100,777664025,True,True
1,16580,05/03/2015,08/03/2015,__016580_2015-03-12 09-09-15.bin,102650051_100,522410444,True,True
2,18618,15/03/2015,16/03/2015,__018618_2015-03-20 11-15-03.bin,62980071_100,565693441,True,True
3,18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,52920071_100,692552249,True,True
4,18643,06/03/2015,07/03/2015,__018643_2015-03-13 12-19-47.bin,122650071_100,525756098,True,True


In [40]:
annotations_codes = pd.merge(annotations, wearcodes, on='serflag', how='left')
print(annotations_codes.shape)
annotations_codes.head()

(1440, 12)


,serflag,start_time,end_time,act,act_label,Monitor,Day1,Day2,binFile,fileSize,day1Ready,day2Ready
0,52920071_100,2015-03-01 04:00:00,2015-03-01 04:10:00,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
1,52920071_100,2015-03-01 04:10:00,2015-03-01 04:20:00,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
2,52920071_100,2015-03-01 04:20:00,2015-03-01 04:30:00,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
3,52920071_100,2015-03-01 04:30:00,2015-03-01 04:40:00,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True
4,52920071_100,2015-03-01 04:40:00,2015-03-01 04:50:00,1.0,Sleeping and resting (including sick in bed),18635,01/03/2015,03/03/2015,__018635_2015-03-11 14-06-25.bin,692552249,True,True


## Load and preprocess data

In [64]:
byName = annotations_codes.groupby('binFile')
dfs = {}
for name, fileAnnotations in byName:
    # Load data
    df = loadData(name)
    
    if df is not None:    
        # Add annotations
        #df = addAnnotations(df, fileAnnotations)
        dfs.update(df)

In [66]:
test_name = dfs.keys()[0]
dfs[test_name].head()

,acceleration,anglez,invalid
timestamp,,,
2015-03-15 04:00:00,0.001881,48.412583,0
2015-03-15 04:00:05,0.001864,48.473354,0
2015-03-15 04:00:10,0.001593,48.491323,0
2015-03-15 04:00:15,0.001915,48.497348,0
2015-03-15 04:00:20,0.001773,48.295863,0


In [86]:
for binFile, day in dfs.keys():
    df = dfs[(binFile, day)]
    annotations_group = byName.get_group(binFile)
    #annotations_group_day = annotations_group[annotations_group['Day'+day]]
    for idx in annotations_group.index:
        rStart = annotations_group.loc[idx, 'start_time']
        rEnd = annotations_group.loc[idx, 'end_time']
        df.loc[(df.index < rEnd) & (df.index > rStart), 'Activity'] = annotations_group.loc[idx, 'act_label']

In [85]:
df

,acceleration,anglez,invalid,Activity
timestamp,,,,
2015-03-15 04:00:00,0.001881,48.412583,0,NaN
2015-03-15 04:00:05,0.001864,48.473354,0,Sleeping and resting (including sick in bed)
2015-03-15 04:00:10,0.001593,48.491323,0,Sleeping and resting (including sick in bed)
2015-03-15 04:00:15,0.001915,48.497348,0,Sleeping and resting (including sick in bed)
2015-03-15 04:00:20,0.001773,48.295863,0,Sleeping and resting (including sick in bed)
2015-03-15 04:00:25,0.001455,48.394817,0,Sleeping and resting (including sick in bed)
2015-03-15 04:00:30,0.001930,48.438541,0,Sleeping and resting (including sick in bed)
2015-03-15 04:00:35,0.001517,48.416913,0,Sleeping and resting (including sick in bed)
2015-03-15 04:00:40,0.002082,48.477323,0,Sleeping and resting (including sick in bed)


In [ ]:
def addAnnotations(data, fileAnnotations):
    for row in fileAnnotations[['ddate1', 'stime', 'etime', 'activity', 'filename']].as_matrix():
        rStart = parseDate(row[0].strip() + ' ' + row[1].strip())
        rEnd   = parseDate(row[0].strip() + ' ' + row[2].strip())
        data.loc[(data.index < rEnd) & (data.index > rStart), 'Activity'] = row[3]
        
    return data